In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import subprocess
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
import threading

In [2]:
 df_url =  pd.read_csv("target_urls.csv")
 urls = df_url['url']
 header={'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.35',
 'Referer':'https://cn.bing.com/'}

# 获取urls

In [3]:
def get_url(url,base):
    links=[]
    try:
        # 发送 GET 请求
        response = requests.get(url=url,headers=header)
        # print(response)
        # 检查响应状态码
        if response.status_code == 200:
            # 使用 BeautifulSoup 解析 HTML 内容
            soup = BeautifulSoup(response.text, 'html.parser')
            # 找到页面中的所有链接
            links = soup.find_all('a', href=True)
            # 筛选域名开头的链接
            links = [link.get('href') for link in links if link.get('href') and link.get('href').startswith(base)]
        else:
            print(f"无法访问网站：{url}")
    except Exception as e:
        print(f"发生错误：{e}")
    return links

In [4]:
res_urls=[]
for url in urls:
    # 获取目标网站url
    targets=get_url(url,url)
    res_url=[]
    for target in targets:
        #获取子网站url
        son_url=get_url(target,url)
        #列表降维
        son_url = [sublist for sublist in son_url]
        # 列表拼接
        res_url.extend(son_url)
        # 列表去重
        res_url=list(set(res_url))
        if(len(res_url)>300): break
    # 元素去重
    print(len(res_url))
    res_urls.append(res_url)

# 创建 DataFrame
df = pd.DataFrame({'urls': urls, 'res_urls': res_urls})

# 将 DataFrame 保存到 CSV 文件
df.to_csv('res_urls.csv', index=False)

451


# 捕获流量

In [5]:
# 创建保存流量的目录
data_flow_dir = "./pcap_data"
os.makedirs(data_flow_dir, exist_ok=True)

# 设置Chrome选项
chrome_options = Options()
# chrome_options.add_argument("--headless")  # 如果你不需要浏览器界面，可以添加这个选项

# 初始化Chrome浏览器
driver_path = "./driver/chromedriver"
driver = webdriver.Chrome(executable_path=driver_path, options=chrome_options)
wait = WebDriverWait(driver, 20)  # 设置显式等待，最长等待时间为20秒


# 定义一个函数用于等待页面完全加载
def wait_for_page_load(driver):
    wait.until(lambda d: d.execute_script("return document.readyState") == "complete")


# 访问每个url
for index, row in df.iterrows():
    url = row["urls"]
    res_urls = row["res_urls"]

    # 创建当前URL的流量文件名
    file_name = os.path.join(data_flow_dir, url.split("//")[1] + ".pcap")

    # 启动tshark捕获流量
    tshark_command = [
        "sudo",
        "tshark",
        "-i",
        "wlp4s0",
        "-w",
        file_name,
        "-f",
        f'host {url.split("//")[1]}',
    ]
    tshark_process = subprocess.Popen(tshark_command)

    # 访问主url
    driver.get(url)
    wait_for_page_load(driver)  # 等待页面完全加载

    with tqdm(total=len(res_urls), desc=url, unit="suburl") as pbar:
        # 访问子url
        for res_url in res_urls:
            driver.get(res_url)
            wait_for_page_load(driver)  # 等待页面完全加载
            pbar.update(1)

    tshark_process.kill()

driver.quit()

Running as user "root" and group "root". This could be dangerous.
Capturing on 'wlp4s0'
 ** (tshark:324529) 19:00:28.978709 [Main MESSAGE] -- Capture started.
 ** (tshark:324529) 19:00:28.978860 [Main MESSAGE] -- File: "./data_flow/www.3dmgame.com_.pcap"
https://www.3dmgame.com: 100%|██████████| 451/451 [09:39<00:00,  1.29s/suburl]  


# 解析pcap为json
运行脚本./pcap_to_json.sh

# 